In [1]:
import requests
import pandas as pd
import sqlite3
from datetime import datetime

In [2]:
# 1. Fetch COVID-19 data from API
url = "https://disease.sh/v3/covid-19/countries"
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
else:
    print("Failed to fetch data.")
    exit()

In [3]:
# 2. Convert to DataFrame
df = pd.json_normalize(data)

In [4]:
df.columns

Index(['updated', 'country', 'cases', 'todayCases', 'deaths', 'todayDeaths',
       'recovered', 'todayRecovered', 'active', 'critical',
       'casesPerOneMillion', 'deathsPerOneMillion', 'tests',
       'testsPerOneMillion', 'population', 'continent', 'oneCasePerPeople',
       'oneDeathPerPeople', 'oneTestPerPeople', 'activePerOneMillion',
       'recoveredPerOneMillion', 'criticalPerOneMillion', 'countryInfo._id',
       'countryInfo.iso2', 'countryInfo.iso3', 'countryInfo.lat',
       'countryInfo.long', 'countryInfo.flag'],
      dtype='object')

In [5]:
# 4. Add timestamp column
df['fetch_date'] = datetime.now().strftime('%Y-%m-%d')

In [6]:
# 5. Save to SQLite database
# Connect to DB
conn = sqlite3.connect("covid_data.db")
cursor = conn.cursor()

# Create table if not exists
cursor.execute('''
    CREATE TABLE IF NOT EXISTS daily_data (
        country TEXT,
        cases INTEGER,
        todayCases INTEGER,
        deaths INTEGER,
        todayDeaths INTEGER,
        recovered INTEGER,
        todayRecovered INTEGER,
        active INTEGER,
        critical INTEGER,
        population INTEGER,
        fetch_date TEXT
    )
''')

# Check for today's entry
fetch_date = df['fetch_date'].iloc[0]
cursor.execute("SELECT COUNT(*) FROM daily_data WHERE fetch_date = ?", (fetch_date,))
if cursor.fetchone()[0] > 0:
    print("✅ Data for today already exists. Skipping insertion.")
    conn.close()
else:
    # Insert only if today's data is not present
    # Keep only required columns (to avoid extra ones like 'updated')
    required_columns = [
        'country', 'cases', 'todayCases', 'deaths', 'todayDeaths',
        'recovered', 'todayRecovered', 'active', 'critical', 'population', 'fetch_date'
    ]
    df = df[required_columns]

    df.to_sql('daily_data', conn, if_exists='append', index=False)
    conn.close()
    print("✅ New data inserted.")
    
print("Completed ✅✅")

✅ New data inserted.
Completed ✅✅


In [14]:

# Save to Excel
df.to_excel("D:\Data\Projects\Covid data\covid_data_exported.xlsx", index=False, sheet_name='Covid_data')

print("Data exported successfully to 'covid_data_exported.xlsx'")

Data exported successfully to 'covid_data_exported.xlsx'


<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\steff\AppData\Local\Temp\ipykernel_7752\3818754135.py:2: SyntaxWarning: invalid escape sequence '\D'
  df.to_excel("D:\Data\Projects\Covid data\covid_data_exported.xlsx", index=False, sheet_name='Covid_data')


,country,cases,todayCases,deaths,todayDeaths,recovered,todayRecovered,active,critical,population,fetch_date
0,Afghanistan,234174,0,7996,0,211080,0,15098,0,40754388,2025-07-19
1,Albania,334863,0,3605,0,330233,0,1025,0,2866374,2025-07-19
2,Algeria,272010,0,6881,0,183061,0,82068,0,45350148,2025-07-19
3,Andorra,48015,0,165,0,0,0,47850,0,77463,2025-07-19
4,Angola,107327,0,1937,0,103419,0,1971,0,35027343,2025-07-19
...,...,...,...,...,...,...,...,...,...,...,...
226,Wallis and Futuna,3550,0,8,0,438,0,3104,0,10982,2025-07-19
227,Western Sahara,10,0,1,0,9,0,0,0,626161,2025-07-19
228,Yemen,11945,0,2159,0,9124,0,662,0,31154867,2025-07-19
229,Zambia,349304,0,4069,0,341316,0,3919,0,19470234,2025-07-19
